In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
import multiprocessing
from pyspark.sql.functions import asc, desc, col, sum, mean, concat, lit, regexp_extract, regexp_replace, when, max, min

Create a jupyter notebook or python script named spark101 for this exercise.

### 1. Create a spark data frame that contains your favorite programming languages.
- The name of the column should be language


In [5]:
pandas_dataframe = pd.DataFrame({
    "Python": np.random.choice([0,1], 20),
    "C++": np.random.choice([0,1], 20),
    "Javascript": np.random.choice([0,1], 20),
})

spark = pyspark.sql.SparkSession.builder.getOrCreate()
df = spark.createDataFrame(pandas_dataframe)

- View the schema of the dataframe

In [7]:
df.schema

StructType(List(StructField(Python,LongType,true),StructField(C++,LongType,true),StructField(Javascript,LongType,true)))

- Output the shape of the dataframe

In [14]:
df.toPandas().shape

(20, 3)

- Show the first 5 records in the dataframe

In [15]:
df.show(5)

+------+---+----------+
|Python|C++|Javascript|
+------+---+----------+
|     0|  0|         1|
|     0|  0|         0|
|     1|  0|         1|
|     1|  1|         1|
|     0|  1|         1|
+------+---+----------+
only showing top 5 rows



### 2. Load the mpg dataset as a spark dataframe.

In [16]:
from pydataset import data

In [17]:
mpg = data('mpg')

In [19]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [20]:
df = spark.createDataFrame(mpg)
df.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

- a. Create 1 column of output that contains a message like the one below:
        The 1999 audi a4 has a 4 cylinder engine.

In [27]:
df.select(concat(lit('The '),df.year,lit(' '),df.manufacturer,lit(' '),df.model,lit(' has a '),df.cyl,lit(' cylinder engine'))).show(truncate=False)

+-----------------------------------------------------------------------------+
|concat(The , year,  , manufacturer,  , model,  has a , cyl,  cylinder engine)|
+-----------------------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
|The 2008 audi a4 has a 6 cylinder engine                                     |
|The 1999 audi a4 quattro has a 4 cylinder engine                             |
|The 1999 audi a4 quattro has a 4 cylinder engine                             |
|The 2008 audi a4 quattro has a 4 cylind

- For each vehicle.

- b. Transform the trans column so that it only contains either manual or auto.

In [29]:
df.select('trans').show()

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
|manual(m5)|
|  auto(av)|
|manual(m5)|
|  auto(l5)|
|manual(m6)|
|  auto(s6)|
|  auto(l5)|
|manual(m5)|
|  auto(s6)|
|manual(m6)|
|  auto(l5)|
|  auto(s6)|
|  auto(s6)|
|  auto(l4)|
|  auto(l4)|
+----------+
only showing top 20 rows



In [44]:
df.where(col('trans').like("%auto%")).select(df.trans).show()

+--------+
|   trans|
+--------+
|auto(l5)|
|auto(av)|
|auto(l5)|
|auto(av)|
|auto(l5)|
|auto(s6)|
|auto(l5)|
|auto(s6)|
|auto(l5)|
|auto(s6)|
|auto(s6)|
|auto(l4)|
|auto(l4)|
|auto(l4)|
|auto(l4)|
|auto(l4)|
|auto(l4)|
|auto(s6)|
|auto(l4)|
|auto(l4)|
+--------+
only showing top 20 rows



In [53]:
df.select(df.trans,
         when(df.trans.like('auto%'), 'auto')
         .otherwise('manual')
         .alias('transmission')
         ).show()

+----------+------------+
|     trans|transmission|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(av)|        auto|
|manual(m5)|      manual|
|  auto(l5)|        auto|
|manual(m6)|      manual|
|  auto(s6)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(s6)|        auto|
|manual(m6)|      manual|
|  auto(l5)|        auto|
|  auto(s6)|        auto|
|  auto(s6)|        auto|
|  auto(l4)|        auto|
|  auto(l4)|        auto|
+----------+------------+
only showing top 20 rows



### 3. Load the tips dataset as a spark dataframe.

In [54]:
tips = data('tips')
df = spark.createDataFrame(tips)
df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



- a. What percentage of observations are smokers?

In [61]:
df.toPandas().shape

(244, 7)

In [66]:
(df.where(df.smoker == 'Yes').count()) / (df.toPandas().shape[0]) * 100

38.114754098360656

- b. Create a column that contains the tip percentage

In [73]:
df.select(
    (df.tip / df.total_bill)
    .alias('tip_percentage')
).show()

+-------------------+
|     tip_percentage|
+-------------------+
|0.05944673337257211|
|0.16054158607350097|
|0.16658733936220846|
| 0.1397804054054054|
|0.14680764538430255|
|0.18623962040332148|
|0.22805017103762829|
|0.11607142857142858|
|0.13031914893617022|
| 0.2185385656292287|
| 0.1665043816942551|
|0.14180374361883155|
|0.10181582360570687|
|0.16277807921866522|
|0.20364126770060686|
|0.18164967562557924|
| 0.1616650532429816|
|0.22774708410067526|
|0.20624631703005306|
|0.16222760290556903|
+-------------------+
only showing top 20 rows



- c. Calculate the average tip percentage for each combination of sex and smoker.

In [82]:
col1 = df.select((df.tip / df.total_bill).alias('tip_percentage'))

In [83]:
col1.show()

+-------------------+
|     tip_percentage|
+-------------------+
|0.05944673337257211|
|0.16054158607350097|
|0.16658733936220846|
| 0.1397804054054054|
|0.14680764538430255|
|0.18623962040332148|
|0.22805017103762829|
|0.11607142857142858|
|0.13031914893617022|
| 0.2185385656292287|
| 0.1665043816942551|
|0.14180374361883155|
|0.10181582360570687|
|0.16277807921866522|
|0.20364126770060686|
|0.18164967562557924|
| 0.1616650532429816|
|0.22774708410067526|
|0.20624631703005306|
|0.16222760290556903|
+-------------------+
only showing top 20 rows



In [86]:
df.groupBy('sex','smoker').agg(mean((df.tip / df.total_bill).alias('tip_percentage'))).show()

+------+------+-------------------------------------------+
|   sex|smoker|avg((tip / total_bill) AS `tip_percentage`)|
+------+------+-------------------------------------------+
|  Male|    No|                         0.1606687151291298|
|  Male|   Yes|                         0.1527711752024851|
|Female|    No|                         0.1569209707691836|
|Female|   Yes|                        0.18215035269941035|
+------+------+-------------------------------------------+



### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.
- Convert the temperatures to farenheight.

- Which month has the most rain, on average?

- Which year was the windiest?

- What is the most frequent type of weather in January?

- What is the average high and low temperature on sunny days in July in 2013 and 2014?

- What percentage of days were rainy in q3 of 2015?

- For each year, find what percentage of days it rained (had non-zero precipitation).